In [ ]:
from numba import jit
import numpy as np

In [ ]:
@jit(nopython=True, cache=True, fastmath=True)
def make_sphere_array(radius):
    arr = np.full((2*radius+1,2*radius+1,2*radius+1), False)
    x0, y0, z0 = int(arr.shape[0]/2),int(arr.shape[1]/2),int(arr.shape[2]/2)
    for x in range(x0-radius, x0+radius+1):
        for y in range(y0-radius, y0+radius+1):
                for z in range(z0-radius, z0+radius+1):
                    deb = np.linalg.norm(np.array([x0-x, y0-y, z0-z], dtype=np.float32))
                    arr[x,y,z] = True if deb <= radius else False
    return arr

def search_light_analysis(data, radius, search_area=None, post_func=None):
    search_area = data if search_area is None else search_area 
    post_func = (lambda x:x) if post_func is None else post_func
    sphere_mask = make_sphere_array(radius)
    for x0 in range(radius, data.shape[0]-radius):
        for y0 in range(radius, data.shape[1]-radius):
            for z0 in range(radius, data.shape[2]-radius):
                if search_area[x0,y0,z0]:
                    target_area = data[x0-radius:x0+radius+1,y0-radius:y0+radius+1,z0-radius:z0+radius+1].copy()
                    available_mask = (target_area.sum(axis=3)!=0) & sphere_mask
                    if np.any(available_mask):
                        values = target_area[available_mask]
                        result = post_func(values)
                        yield result, (x0,y0,z0)

In [ ]:
# Load brain data
beta = np.load('Sample_fMRI_data.npz')['beta']

# Define mask
mask = np.ones((beta.shape[:3]))
tmp = np.sum((beta!=0),axis=3)
mask[(tmp<16)] = 0

# Searchlight
brain_inform_list = [inform for inform in search_light_analysis(beta,radius=3,search_area=mask)]
nvv = len(brain_inform_list) # number of valid voxels
brain_rdm = np.zeros((nvv,16,16))
cnt_info = np.zeros((nvv),dtype=int)
for vi in range(nvv): 
    tp = brain_inform_list[vi][0] # nvv,192
    tp2 = 1-np.corrcoef(tp, rowvar=False)
    brain_rdm[vi] = tp2
    cnt_info[vi] = tp.shape[0]
    
# Save brain_rdm
savepath = 'Sample_Neural_RDM.npz'
np.savez(savepath, **{'rdm':brain_rdm, 'info_list':brain_inform_list})